# Genetic Algorithm for Synthesizer Design Optimization 🧬🎹

---

Welcome to this Jupyter Notebook where we will explore the fascinating intersection of genetics, computer science, and sound design! 💻🔊

Our objective? To develop an implementation of a genetic algorithm that's capable of optimizing synthesizer design parameters. By doing so, we aim to generate a synthesized sound that closely matches a given target sound. 🎯

The world of sound synthesis is deeply complex and nuanced, with a myriad of parameters that can be manipulated to create unique sounds. From the frequency of an oscillator to the attack time of an envelope, each parameter shapes the resulting sound in its own way. 🎛️🎚️

And, here's where the genetic algorithms come in. Inspired by natural evolution, genetic algorithms are a search heuristic that are particularly well-suited for black box optimization problems, such as the one we are tackling here. 🏞️🔍

Just like natural selection favors the fittest individuals, our algorithm will favor synthesizer configurations that produce a sound closest to our target sound. Over many generations, the algorithm will "evolve" the parameters to improve the fitness of our synthesized sound. 🔄🧬

For our demo, we're going to use a unique, expressive piano sample from the track 'Blue in Green' by the legendary jazz pianist, Bill Evans. 🎶🎹🎼

As we proceed, we'll detail each step of the process, from defining the genetic algorithm, to setting up the synthesizer, selecting the fitness function, and more. We hope this guide serves as a useful resource for anyone interested in combining the power of genetics, artificial intelligence, and sound design. 📚🤓

Let's dive in!

## Imports

In [37]:
import numpy as np
import dawdreamer
import sys; sys.path.append('../')
import os
import IPython.display as ipd
import pandas as pd
import torch
from librosa import load
from librosa.feature import mfcc
from src.utils import *

In [38]:
# print the ../ path in full as a check
print(os.path.abspath('../'))

/Users/malek8/My Drive (malek8@mit.edu)/MIT/Spring 2023/4.453 (Creative Machine Learning for Design)/GASS Term Project/gass_repo


## Load the Target Sound

In [3]:
# specify the folder of the target sound
target_folder = '../timbre-exp/target-dataset'
# print the absolute path of the target folder
print(f'Absolute Path: {os.path.abspath(target_folder)}')

Absolute Path: /Users/malek8/My Drive (malek8@mit.edu)/MIT/Spring 2023/4.453 (Creative Machine Learning for Design)/GASS Term Project/gass_repo/timbre-exp/target-dataset


In [6]:
# specify the target sound file base name
target_file_name = 'bill-evans-piano'

# combine the target folder and target file name to get the target file path
target_file_path = os.path.join(target_folder,target_file_name) + '.wav'

print(f'Target File Path: {os.path.abspath(target_file_path)}')

Target File Path: /Users/malek8/My Drive (malek8@mit.edu)/MIT/Spring 2023/4.453 (Creative Machine Learning for Design)/GASS Term Project/gass_repo/timbre-exp/target-dataset/bill-evans-piano.wav


In [36]:
# read the wav file
target_audio, target_sample_rate = load(target_file_path, sr=44100)
print(f'Target Sample Rate: {target_sample_rate}')
print(f'Target Audio Shape: {target_audio.shape}')

Target Sample Rate: 44100
Target Audio Shape: (44100,)


In [20]:
# play the target audio
ipd.Audio(target_audio,rate=target_sample_rate)

## Obtain Feature Representation of Target Sound

In [24]:
# apply the MFCC transform to the target audio
target_mfcc = mfcc(y=target_audio,sr=target_sample_rate).reshape(-1)
print(f'Target MFCC Shape: {target_mfcc.shape}')

Target MFCC Shape: (1740,)


## Find Closest Feature Match in Preset Dataset $\implies$ Obtain $p^* \in \mathbb{R}^{k \times 1}$

---

Here $k$ denotes the number of parameters in the vector that controls the output of the synthesizer.

### Load the Preset Dataset

In [30]:
# load the preset dataset
dataset = pd.DataFrame(torch.load('../dataset/preset_dataset_musicnn.pt'))
print(f'Dataset Shape: {dataset.shape}')


Dataset Shape: (367, 6)


In [31]:
# print the first 5 rows of the dataset
dataset.head(5)

,preset_names,parameters,parameters_names,mapped_parameter_names,raw_audio,musicnn_features
0,Default,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...","[@modulation, @dcolfovalue, @dcopwmvalue, @dco...","[{'match': 'modulation', 'value': 0.0, 'index'...","{'C2': [tensor(-3.1763e-06), tensor(-0.0001), ...","{'C2': [tensor(-0.0003), tensor(-0.6242), tens..."
1,MT Tailwhip Organ (2),"[0.0, 0.0728217363357544, 0.4655496180057526, ...","[@modulation, @dcolfovalue, @dcopwmvalue, @dco...","[{'match': 'modulation', 'value': 0.0, 'index'...","{'C2': [tensor(1.1023e-05), tensor(2.2758e-05)...","{'C2': [tensor(-0.0003), tensor(-0.6242), tens..."
2,Super Jumper,"[0.0, 0.540000021, 0.444000036, 0.0, 0.0, 0.0,...","[@dcolfovalue, @dcopwmvalue, @dcopwmmode, @dco...","[{'match': 'dco lfo value', 'value': 0.0, 'ind...","{'C2': [tensor(-1.5057e-05), tensor(4.3656e-05...","{'C2': [tensor(-0.0003), tensor(-0.6242), tens..."
3,Nice Filter Sweep,"[0.0, 0.540000021, 0.444000036, 0.0, 0.0, 0.0,...","[@dcolfovalue, @dcopwmvalue, @dcopwmmode, @dco...","[{'match': 'dco lfo value', 'value': 0.0, 'ind...","{'C2': [tensor(-0.0130), tensor(0.0137), tenso...","{'C2': [tensor(-0.0003), tensor(-0.6242), tens..."
4,Chillin R2D2,"[0.0, 0.540000021, 0.444000036, 0.0, 0.0, 0.0,...","[@dcolfovalue, @dcopwmvalue, @dcopwmmode, @dco...","[{'match': 'dco lfo value', 'value': 0.0, 'ind...","{'C2': [tensor(0.0289), tensor(0.1960), tensor...","{'C2': [tensor(-0.0003), tensor(-0.6242), tens..."


In [33]:
# Define constants related to the TAL-Uno plugin
PRESET_FOLDER = "/Users/malek8/Library/Application Support/ToguAudioLine/TAL-U-No-LX/presets"
PRESET_EXT = ".pjunoxl"

In [35]:
# obtain the first row of the dataset
row = dataset.iloc[0]

# index into the parameters column
parameters = row['parameters']

# find the max and minimum values of the parameters
max_value = np.max(parameters)
min_value = np.min(parameters)

# print the max and min
print(f'Max Value: {max_value}')
print(f'Min Value: {min_value}')

Max Value: 1.0
Min Value: 0.0


In [34]:
# obtain a list of all the preset paths
preset_paths = []
for root, dirs, files in os.walk(PRESET_FOLDER):
    for file in files:
        if file.endswith(PRESET_EXT):
            preset_paths.append(os.path.join(root, file))
print(f'The first 5 preset paths:\n{preset_paths[:5]}')

The first 5 preset paths:
['/Users/malek8/Library/Application Support/ToguAudioLine/TAL-U-No-LX/presets/Default.pjunoxl', '/Users/malek8/Library/Application Support/ToguAudioLine/TAL-U-No-LX/presets/MT Tailwhip Organ (2).pjunoxl', '/Users/malek8/Library/Application Support/ToguAudioLine/TAL-U-No-LX/presets/TAL Presets Bank/FX/Super Jumper.pjunoxl', '/Users/malek8/Library/Application Support/ToguAudioLine/TAL-U-No-LX/presets/TAL Presets Bank/FX/Nice Filter Sweep.pjunoxl', '/Users/malek8/Library/Application Support/ToguAudioLine/TAL-U-No-LX/presets/TAL Presets Bank/FX/Chillin R2D2.pjunoxl']


### Find Closest Match Based on Minimium Euclidean Distance 

In [40]:
top10presets = find_closest_preset_from_mfcc(target_audio, target_sample_rate, dataset)
print(f'The top 10 presets:\n{top10presets}')

Likely note: F4, likely frequency: 349.1941058508811
The top 10 presets:
['WND Flute 1 FMR' 'CHO Voice Chorus FMR' 'BAS Pulse Bass 1 FMR'
 'ORG Organ 3 FMR' 'ORG Another Organ FMR' 'BAS Round Bass 2 FMR'
 'DRM Synth Drum 1 FMR' 'MT Tailwhip Organ (2)' 'BAS Synth Bass 1 FMR'
 'DRM Synth Drum 3 FMR']


In [43]:
# find the path of the top preset in top10presets using preset_paths
top_preset_path = [x for x in preset_paths if top10presets[0] in x][0]

# print the top preset path
print(f'Top Preset Path: {top_preset_path}')


Top Preset Path: /Users/malek8/Library/Application Support/ToguAudioLine/TAL-U-No-LX/presets/FMR Presets Bank/Wind/WND Flute 1 FMR.pjunoxl


## Apply Genetic Algorithm $\implies$ Obtain $x^*$

---

Here, we seek to apply the genetic algorithm on our initial population obtained in the previous step to obtain $x^*$, the optimal synthesizer preset configuration that produces a sound most similar to the target input sound $y \in \mathbb{R}^{n \times 1}$.

In [44]:
# define some constants related to dawdreamer loading
BUFFER_SIZE = 128
SYNTH_PLUGIN_PATH = "/Library/Audio/Plug-Ins/VST3/TAL-U-NO-LX-V2.vst3"
SYNTH_NAME = "TAL-Uno"

In [45]:
# initialize the DawDreamer engine
engine = dawdreamer.RenderEngine(sample_rate=target_sample_rate, block_size=BUFFER_SIZE)

In [46]:
# initialize the DawDreamer plugin
plugin = load_plugin_with_dawdreamer(SYNTH_PLUGIN_PATH,SYNTH_NAME,engine)

error: attempt to map invalid URI `/Library/Audio/Plug-Ins/VST3/TAL-U-NO-LX-V2.vst3'


In [ ]:
x_star = optimize_preset_with_ga_mfcc(top_preset_path,plugin,engine,target_mfcc,ga_settings)

## Evaluate Sounds Produced by $x^*$